# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load results from part 1 of analysis
cities_results_df = pd.DataFrame(pd.read_csv('../output_data/city_data.csv'))
cities_results_df

,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Windspeed,Country,Date
0,Nanortalik,52.900587,-36.458261,52.18,66,98,14.74,GL,1595369392
1,Baykit,62.405033,96.822185,54.75,71,100,1.95,RU,1595369392
2,Clyde River,72.839069,-73.057257,44.60,93,90,6.93,CA,1595369392
3,Punta Arenas,-85.252199,-114.347033,28.40,86,13,4.70,CL,1595369392
4,Beira,-21.405820,36.711405,69.80,77,3,9.17,MZ,1595369392
...,...,...,...,...,...,...,...,...,...
571,Comodoro Rivadavia,-44.298844,-68.866486,39.20,86,40,5.82,AR,1595369483
572,Yaring,7.804956,101.553789,77.45,87,97,4.63,TH,1595369483
573,Juneau,58.743614,-135.948024,62.60,72,75,4.70,US,1595369287
574,Saint-Georges,3.598226,-51.294872,87.80,66,20,11.41,GD,1595369483


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#mapping the cities with humidity weighted heatmap
locations = cities_results_df[['Lat', 'Lng']]
humidity = cities_results_df['Humidity']

fig = gmaps.figure(center=(10.0, 0.0), zoom_level=1.9,
                  layout={
        'width': '1000px',
        'height': '600px',
        'padding': '3px',
        'border': '4px solid black'
        })
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, max_intensity=0.1, point_radius=30.0))
display(fig)


Figure(layout=FigureLayout(border='4px solid black', height='600px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#filter results by criteria
vacation_df = cities_results_df.loc[(cities_results_df['Max Temperature']< 80) &
                                   (cities_results_df['Max Temperature']> 70) &
                                   (cities_results_df['Windspeed']< 10) &
                                   (cities_results_df['Cloudiness'] == 0)]

vacation_df = vacation_df.dropna()
vacation_df

,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Windspeed,Country,Date
86,Lusambo,-4.416916,23.120402,71.10,40,0,1.16,CD,1595369406
88,Komsomolskiy,67.836958,175.195193,71.60,43,0,9.17,UZ,1595369406
99,Arraial Do Cabo,-32.129768,-29.526280,71.60,88,0,8.05,BR,1595369408
257,Alcudia,40.524396,3.510783,77.00,57,0,4.70,ES,1595369433
271,Nioki,-3.211677,18.356043,77.97,56,0,2.08,CD,1595369435
288,Eregli,37.216455,33.967447,72.16,44,0,6.53,TR,1595369438
328,Sanary-Sur-Mer,41.619662,5.431765,77.00,79,0,1.99,FR,1595369445
331,Jaciara,-16.045144,-55.413237,76.17,35,0,3.22,BR,1595369443
372,Ribas Do Rio Pardo,-20.578416,-53.869038,75.43,31,0,4.94,BR,1595369452
377,Taguatinga,-12.893242,-46.253716,77.97,44,0,7.63,BR,1595369453


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#prepare hotel dataframe for hotel name and location
hotel_df = vacation_df.copy()
hotel_df['Hotel Name'] = None
hotel_df['Hotel Lat'] = None
hotel_df['Hotel Lng'] = None
hotel_df= hotel_df.reset_index(drop= True)

In [6]:
# iterate through the hotel dataframe and call google api to find hotels nearby.
#Increasing radius to 15,000 meters for more selections.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    target_coordinates = f'{row[1]}, {row[2]}'
    target_radius = 15000
    target_type = "hotel"
    target_search = "hotel"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
        
    try:
        re = requests.get(base_url, params=params)
        response = re.json()
        
        #add names and locations to dataframe for mapping later
        hotel_df['Hotel Name'].values[index] = response['results'][0]['name']
        hotel_df['Hotel Lat'].values[index] = response['results'][0]['geometry']['location']['lat']
        hotel_df['Hotel Lng'].values[index] = response['results'][0]['geometry']['location']['lng']
    except IndexError:
        pass
    

In [7]:
#dataframe displaying places with hotels near 5000 meters of location.
hotel_df

,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
0,Lusambo,-4.416916,23.120402,71.10,40,0,1.16,CD,1595369406,None,None,None
1,Komsomolskiy,67.836958,175.195193,71.60,43,0,9.17,UZ,1595369406,None,None,None
2,Arraial Do Cabo,-32.129768,-29.526280,71.60,88,0,8.05,BR,1595369408,None,None,None
3,Alcudia,40.524396,3.510783,77.00,57,0,4.70,ES,1595369433,None,None,None
4,Nioki,-3.211677,18.356043,77.97,56,0,2.08,CD,1595369435,None,None,None
5,Eregli,37.216455,33.967447,72.16,44,0,6.53,TR,1595369438,None,None,None
6,Sanary-Sur-Mer,41.619662,5.431765,77.00,79,0,1.99,FR,1595369445,None,None,None
7,Jaciara,-16.045144,-55.413237,76.17,35,0,3.22,BR,1595369443,None,None,None
8,Ribas Do Rio Pardo,-20.578416,-53.869038,75.43,31,0,4.94,BR,1595369452,Hotel Ideal,-20.4573,-53.7577
9,Taguatinga,-12.893242,-46.253716,77.97,44,0,7.63,BR,1595369453,None,None,None


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

#copied so I could test without running Google API a million times :)

hotel_df2 = hotel_df.copy()
hotel_df2 = hotel_df2.dropna()

hotel_info = [info_box_template.format(**row) for index, row in hotel_df2.iterrows()]
locations = hotel_df2[["Hotel Lat", "Hotel Lng"]]

#dropped None and iterated tuples casting into a list for the heatmap
locations = list(locations.itertuples(index=False, name=None))

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

#add Hotel Name, City and County when clicked in the map
symbol_layer = gmaps.symbol_layer(
            locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(symbol_layer)
# Display figure
display(fig)

Figure(layout=FigureLayout(border='4px solid black', height='600px', padding='3px', width='1000px'))